In [1]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)


## Lecture 12 ##

## Activity ##

In [2]:
# From the CORGIS Dataset Project
# By Austin Cory Bart acbart@vt.edu
# Version 2.0.0, created 3/22/2016
# https://corgis-edu.github.io/corgis/csv/skyscrapers/

sky = Table.read_table('skyscrapers.csv')
sky = (sky.with_column('age', 2022 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

name,material,city,height,age
One World Trade Center,mixed/composite,New York City,541.3,8
Willis Tower,steel,Chicago,442.14,48
432 Park Avenue,concrete,New York City,425.5,7


1. For each city, what’s the tallest building for each material?

In [5]:
# 1. For each city, what’s the tallest building for each material?

sky.pivot('material','city',values = 'height',collect = max)

city,concrete,mixed/composite,steel
Atlanta,264.25,311.8,169.47
Austin,208.15,0,93.6
Baltimore,161.24,0,155.15
Boston,121.92,139,240.79
Charlotte,265.48,239.7,179.23
Chicago,423.22,306.94,442.14
Cincinnati,125,202.69,175
Cleveland,125,288.65,215.8
Columbus,79.25,0,169.3
Dallas,176.48,280.72,270.06


2. For each city, what’s the height difference between the tallest steel building and the tallest concrete building?

In [7]:
# 2. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?

by_city = sky.pivot('material','city',values = 'height',collect = max)

by_city.with_columns('steel - concrete',by_city.column('steel') - by_city.column('concrete'))


city,concrete,mixed/composite,steel,steel - concrete
Atlanta,264.25,311.8,169.47,-94.78
Austin,208.15,0,93.6,-114.55
Baltimore,161.24,0,155.15,-6.09001
Boston,121.92,139,240.79,118.87
Charlotte,265.48,239.7,179.23,-86.25
Chicago,423.22,306.94,442.14,18.92
Cincinnati,125,202.69,175,50
Cleveland,125,288.65,215.8,90.8
Columbus,79.25,0,169.3,90.05
Dallas,176.48,280.72,270.06,93.58


3. Generate a table of the names of the oldest buildings for each material for each city:

In [ ]:
# 3. Generate a table of the names of the oldest buildings for each 
#    material for each city:

# Hint: You can use sort to find the name of the oldest building in the dataset
sky.sort('age', descending=True).column('name').item(0)


# Put your solution here








(No peeking! A solution appears below.)

In [ ]:
def first(s):
    "Return the first element in an array."
    return s.item(0)

(sky
 .sort('age', descending=True)
 .pivot('material', 'city', 'name', first)
)

## Joins ##

In [8]:
drinks = Table(['Drink','Cafe','Price'])
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

Drink,Cafe,Price
Milk Tea,Asha,5.5
Espresso,Strada,1.75
Latte,Strada,3.25
Espresso,FSM,2


In [9]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

Coupon % off,Location
10,Asha
25,Strada
5,Asha


In [10]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

Cafe,Drink,Price,Coupon % off
Asha,Milk Tea,5.5,10
Asha,Milk Tea,5.5,5
Strada,Espresso,1.75,25
Strada,Latte,3.25,25


In [11]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
combined.with_column(
    'Discounted Price', 
    combined.column('Price') * discounted_frac
)

Cafe,Drink,Price,Coupon % off,Discounted Price
Asha,Milk Tea,5.5,10,4.95
Asha,Milk Tea,5.5,5,5.225
Strada,Espresso,1.75,25,1.3125
Strada,Latte,3.25,25,2.4375


## Another Join Example ##

In [12]:
full = Table.read_table('nc-est2019-agesex-res.csv')
census = full.select('SEX', 'AGE', 'POPESTIMATE2019')
census.show(3)

SEX,AGE,POPESTIMATE2019
0,0,3783052
0,1,3829599
0,2,3922044


In [13]:
sex_codes = Table().with_columns(
    'SEX CODE', make_array(0, 1, 2),
    'CODE DEFINITION', make_array('All', 'Selected Male', 'Selected Female')
)
sex_codes

SEX CODE,CODE DEFINITION
0,All
1,Selected Male
2,Selected Female


In [15]:
census.join('SEX', sex_codes, 'SEX CODE').sort('AGE').show(6)

SEX,AGE,POPESTIMATE2019,CODE DEFINITION
0,0,3783052,All
1,0,1935117,Selected Male
2,0,1847935,Selected Female
0,1,3829599,All
1,1,1958585,Selected Male
2,1,1871014,Selected Female


## Bike Sharing ##

In [ ]:
trip = Table.read_table('trip.csv')
trip.show(3)

## Distribution of Durations ##

In [ ]:
trip.hist('Duration')

In [ ]:
trip.sort('Duration', descending=True)

In [ ]:
commute = trip.where('Duration', are.below(1800))
commute.hist('Duration')

In [ ]:
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

In [ ]:
# Approx percent of people who have 
# a ride duration between 250 and 500 seconds
# "between" = [250, 500) 

(500-250) * 0.15 

In [ ]:
commute.where('Duration', are.between(250, 500)).num_rows

In [ ]:
commute.num_rows

In [ ]:
129079 / 338343

In [ ]:
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

In [ ]:
commute.hist('Duration', bins=60, unit='Second')

## Start and End Stations ##

In [ ]:
# Most common start station

starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
# Numbers of trips between stations

commute.pivot('Start Station', 'End Station')

In [ ]:
# Average durations of trips between stations

commute.pivot('Start Station', 'End Station', values='Duration', collect=np.average)

## Activities ##

**Activity 1:** Find the fastest trip ever between each pair of stations? You might find the following table useful.

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
shortest = ...

**Activity 2:** Estimate the 5 stations closest to Civic Center BART by minimum trip time. *Hint* The Civic Center BART is listed as `Civic Center BART (7th at Market)` in the `shortest` table.

In [ ]:
from_cc = ...